In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.ml import PipelineModel
from pyspark.ml.classification import LinearSVCModel

In [0]:
USERNAME = "big-data"
PASSWORD = "rZ5PCb8ugPiuCm3xCW1oycmH01VNWE"

In [0]:
df = spark.readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ctvdvd2dt8sjp3bod5rg.any.eu-central-1.mpx.prd.cloud.redpanda.com:9092") \
  .option("kafka.sasl.jaas.config","kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username='{}' password='{}';".format(USERNAME, PASSWORD)) \
  .option("kafka.ssl.endpoint.identification.algorithm", "https") \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-512") \
  .option("subscribe", "project") \
  .option("startingOffsets", "earliest") \
  .load()

In [0]:
df.select('*')

def funcao(df, epoch_id):
    print(df.count())

In [0]:
query = df\
    .writeStream\
    .foreachBatch(funcao)\
    .start()

In [0]:
schema = StructType([
    StructField("currency", IntegerType(), True),
    StructField("city_size", IntegerType(), True),
    StructField("card_present", StringType(), True),
    StructField("channel", IntegerType(), True),
    StructField("distance_from_home", IntegerType(), True),
    StructField("amounteuro", FloatType(), True),
    StructField("time", IntegerType(), True),
    StructField("is_fraud", StringType(), True)
])

In [0]:
df_parsed = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

In [0]:
pipelineModel = PipelineModel.load(path='dbfs:/FileStore/Trabalho/Projeto_Fraud/pipeline_model_LR')
model = pipelineModel.transform(df_parsed)
dataset = model.select("features_2")

In [0]:
model = LinearSVCModel.load(path='dbfs:/FileStore/Trabalho/ML Streaming/best_svc_model')

In [0]:
predictions = model.transform(dataset)
selected = predictions.select('*')
query = selected \
    .writeStream\
    .outputMode('append')\
    .format('console')\
    .start()
query.awaitTermination()